In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql
from datetime import datetime, timedelta
import numpy as np

import pandas as pd
from dotenv import load_dotenv

load_dotenv()

HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": "localhost",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}



## Buscando dados da tabela 'ml_fulfillment' para alimentar tabela 'fulfillment_stock'

In [ ]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM ml_fulfillment"

df = pd.read_sql(sql_query, conn)

In [ ]:
print(df.shape)
print(df.columns)

In [ ]:
print(df['ml_inventory_id'].value_counts())
print(len(df['ml_inventory_id'].unique()))

Até aqui:

596 é o total de ml_inventory_id únicos. Ou seja, temos 596 produtos únicos neste caso

In [ ]:
# Filtrando
valor_desejado = 'TNXB03680'
resultado = df.loc[df['ml_inventory_id'] == valor_desejado]
resultado

In [ ]:
df = df.drop_duplicates()
print(df.shape)
print(df['ml_inventory_id'].value_counts())
print(len(df['ml_inventory_id'].unique()))

In [ ]:
# Filtrando
valor_desejado = 'UIUW89800'
resultado = df.loc[df['ml_inventory_id'] == valor_desejado]
resultado

### Filtrando as colunas necessárias

In [ ]:
cols = ['ml_inventory_id', 'available_quantity', 'created_at']
db_fulfillment = df[cols].copy()
db_fulfillment

In [ ]:
duplicates = db_fulfillment.duplicated()

num_duplicates = duplicates.sum()

print("Número de linhas duplicadas:", num_duplicates)

duplicated_rows = db_fulfillment[duplicates]
print("Linhas duplicadas:")
print(duplicated_rows)



In [ ]:
db_fulfillment = db_fulfillment.drop_duplicates()
print(db_fulfillment.shape)
# print(db_fulfillment['ml_inventory_id'].value_counts())
# print(len(db_fulfillment['ml_inventory_id'].unique()))

In [ ]:
db_fulfillment

In [ ]:
db_fulfillment.info()

### Enviando Dados para a tabela 'fulfillment_stock'

In [ ]:

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Itere pelas linhas do DataFrame e insira os dados na tabela
for index, row in db_fulfillment.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO fulfillment_stock (ml_inventory_id, available_quantity, created_at)
        VALUES (%s, %s, %s)
    """)
    cursor.execute(insert_query, (row['ml_inventory_id'], row['available_quantity'], row['created_at']))

# Confirme as alterações
conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()

print('Dados inseridos com sucesso!')


In [ ]:
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Itere pelas linhas do DataFrame e insira os dados na tabela
for index, row in db_fulfillment.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO fulfillment_stock (ml_inventory_id, available_quantity, created_at)
        VALUES (%s, %s, %s)
    """)
    cursor.execute(insert_query, (row['ml_inventory_id'], row['available_quantity'], row['created_at']))

# Confirme as alterações
conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()

print('Dados inseridos com sucesso!')


## Buscando dados de 'fulfillment_stock'

In [6]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM fulfillment_stock"

df_stock = pd.read_sql(sql_query, conn)
df_stock.shape

C:\Users\couti\AppData\Local\Temp\ipykernel_17620\3803223401.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stock = pd.read_sql(sql_query, conn)


(19668, 3)

In [7]:
## verificando quantidade de dias cobertos pela tabela 'fulfillment_stock' ##
df_stock.shape[0] / 596

33.0

In [8]:
df_stock = df_stock.sort_values(by='created_at', ascending=False)

In [9]:
## Criando coluna de data ##

df_stock['data'] = df_stock['created_at'].dt.date

df_stock

,ml_inventory_id,available_quantity,created_at,data
19667,UCQQ19395,0,2023-11-13 12:31:28.615487,2023-11-13
19275,LDBL69476,0,2023-11-13 12:31:28.615487,2023-11-13
19273,EDGX70669,0,2023-11-13 12:31:28.615487,2023-11-13
19272,VFWO70372,0,2023-11-13 12:31:28.615487,2023-11-13
19271,GLMK71067,0,2023-11-13 12:31:28.615487,2023-11-13
...,...,...,...,...
18081,EDGX70669,0,2023-10-10 01:00:00.000000,2023-10-10
18082,KYGE71098,0,2023-10-10 01:00:00.000000,2023-10-10
18083,LDBL69476,0,2023-10-10 01:00:00.000000,2023-10-10
18084,PUCU70522,0,2023-10-10 01:00:00.000000,2023-10-10


In [10]:
# ## verificando onde df_filtrado2['available_quantity'] = 0 ##

# valor_desejado = 0  
# linhas_selecionadas = df[df['available_quantity'] == valor_desejado]

# linhas_selecionadas.head(3)

In [11]:
# ## verificando ml_inventory_id ##

# valor_desejado = 'ZQIV51979'  
# linhas_selecionadas = df[df['ml_inventory_id'] == valor_desejado]

# linhas_selecionadas.head(3)

In [12]:
# ## Mudando valores de 'available_quantity', onde available_quantity=0, available_quantity=1 

# df.loc[df['available_quantity'] == 0, 'available_quantity'] = 1
# df.head(4)

In [13]:
# ## verificando ml_inventory_id ##

# valor_desejado = 'ZQIV51979'  
# linhas_selecionadas = df[df['ml_inventory_id'] == valor_desejado]

# linhas_selecionadas.head(3)

### Filtrando do DF por dias

In [190]:
# não calcular o dia atual
# se hoje 09, calcular d x até 08
days = 11

data_atual = datetime.now().date()  # Extrai a data do datetime de data atual
data_inicio = (data_atual - timedelta(days=days))  # Extrai a data do datetime da data de início

print(data_inicio)
print(data_atual)

# Filtra para incluir apenas as linhas dentro do período desejado
df_filtrado = df_stock[(df_stock['data'] >= data_inicio) & (df_stock['data'] <= data_atual)]


2023-11-02
2023-11-13


In [191]:
df_filtrado.shape

(5960, 4)

In [192]:
df_filtrado['ml_inventory_id'].value_counts()

ml_inventory_id
UCQQ19395    10
UWGL04723    10
KKKP96008    10
NFWV90018    10
POTW78210    10
             ..
PKNN52962    10
TDFV51283    10
FXMN50419    10
RBUB53192    10
EDHV29406    10
Name: count, Length: 596, dtype: int64

In [193]:
df_filtrado.head(4)

,ml_inventory_id,available_quantity,created_at,data
19667,UCQQ19395,0,2023-11-13 12:31:28.615487,2023-11-13
19275,LDBL69476,0,2023-11-13 12:31:28.615487,2023-11-13
19273,EDGX70669,0,2023-11-13 12:31:28.615487,2023-11-13
19272,VFWO70372,0,2023-11-13 12:31:28.615487,2023-11-13


In [74]:
## Cria coluna has_stock, se available_quantity <= 0, has_stock= False ##
df_filtrado = df_filtrado.assign(has_stock=lambda x: x["available_quantity"] > 0)
df_filtrado.head(10)

,ml_inventory_id,available_quantity,created_at,data,has_stock
19667,UCQQ19395,0,2023-11-13 12:31:28.615487,2023-11-13,False
19275,LDBL69476,0,2023-11-13 12:31:28.615487,2023-11-13,False
19273,EDGX70669,0,2023-11-13 12:31:28.615487,2023-11-13,False
19272,VFWO70372,0,2023-11-13 12:31:28.615487,2023-11-13,False
19271,GLMK71067,0,2023-11-13 12:31:28.615487,2023-11-13,False
19270,PNST71108,0,2023-11-13 12:31:28.615487,2023-11-13,False
19269,TKMI71532,0,2023-11-13 12:31:28.615487,2023-11-13,False
19268,SCVF71741,0,2023-11-13 12:31:28.615487,2023-11-13,False
19267,LEZR73049,0,2023-11-13 12:31:28.615487,2023-11-13,False
19266,JMQF70068,0,2023-11-13 12:31:28.615487,2023-11-13,False


In [75]:
df_stock_true = df_filtrado[df_filtrado['has_stock'] == True]
df_stock_true

,ml_inventory_id,available_quantity,created_at,data,has_stock
19262,MENA66694,1,2023-11-13 12:31:28.615487,2023-11-13,True
19261,MHXR70863,3,2023-11-13 12:31:28.615487,2023-11-13,True
19260,ELKX63703,6,2023-11-13 12:31:28.615487,2023-11-13,True
19259,CMXV62869,8,2023-11-13 12:31:28.615487,2023-11-13,True
19258,SPKV63227,7,2023-11-13 12:31:28.615487,2023-11-13,True
...,...,...,...,...,...
4359,CMXV62869,9,2023-11-02 01:00:00.000000,2023-11-02,True
4360,ELKX63703,6,2023-11-02 01:00:00.000000,2023-11-02,True
4361,MHXR70863,3,2023-11-02 01:00:00.000000,2023-11-02,True
4362,MENA66694,1,2023-11-02 01:00:00.000000,2023-11-02,True


In [79]:
ml_id_test = 'MENA66694'
y = df_filtrado[df_filtrado['ml_inventory_id'] == ml_id_test]
d = data_atual-data_inicio

In [89]:
print(f'Quantidade de vezes em que MENA66694 esteve disponível no período de {d.days-1} dias  = {y.shape[0]}')

Quantidade de vezes em que MENA66694 esteve disponível no período de 10 dias  = 10


In [81]:
y

,ml_inventory_id,available_quantity,created_at,data,has_stock
19262,MENA66694,1,2023-11-13 12:31:28.615487,2023-11-13,True
18666,MENA66694,1,2023-11-10 20:22:52.890804,2023-11-10,True
190,MENA66694,1,2023-11-09 16:33:03.451375,2023-11-09,True
786,MENA66694,1,2023-11-08 01:00:00.000000,2023-11-08,True
1382,MENA66694,1,2023-11-07 01:00:00.000000,2023-11-07,True
1978,MENA66694,1,2023-11-06 01:00:00.000000,2023-11-06,True
2574,MENA66694,1,2023-11-05 01:00:00.000000,2023-11-05,True
3170,MENA66694,1,2023-11-04 01:00:00.000000,2023-11-04,True
3766,MENA66694,1,2023-11-03 01:00:00.000000,2023-11-03,True
4362,MENA66694,1,2023-11-02 01:00:00.000000,2023-11-02,True


In [83]:
## Contando dias em que produto esteve disponível ##
days_available = df_filtrado.groupby('ml_inventory_id')['has_stock'].sum().reset_index()
# # Renomeie a coluna resultante para 'days_available'
days_available = days_available.rename(columns={'has_stock': 'days_available'})


In [84]:
days_available.shape

(596, 2)

In [85]:
df_filtrado.shape

(5960, 5)

In [90]:
z = days_available[days_available['ml_inventory_id'] == ml_id_test]
z

,ml_inventory_id,days_available
287,MENA66694,10


In [91]:
z = df_filtrado[df_filtrado['ml_inventory_id'] == ml_id_test]
z

,ml_inventory_id,available_quantity,created_at,data,has_stock
19262,MENA66694,1,2023-11-13 12:31:28.615487,2023-11-13,True
18666,MENA66694,1,2023-11-10 20:22:52.890804,2023-11-10,True
190,MENA66694,1,2023-11-09 16:33:03.451375,2023-11-09,True
786,MENA66694,1,2023-11-08 01:00:00.000000,2023-11-08,True
1382,MENA66694,1,2023-11-07 01:00:00.000000,2023-11-07,True
1978,MENA66694,1,2023-11-06 01:00:00.000000,2023-11-06,True
2574,MENA66694,1,2023-11-05 01:00:00.000000,2023-11-05,True
3170,MENA66694,1,2023-11-04 01:00:00.000000,2023-11-04,True
3766,MENA66694,1,2023-11-03 01:00:00.000000,2023-11-03,True
4362,MENA66694,1,2023-11-02 01:00:00.000000,2023-11-02,True


In [92]:
df_stock_days = df_filtrado.merge(days_available, on='ml_inventory_id', how='left')
df_stock_days

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available
0,UCQQ19395,0,2023-11-13 12:31:28.615487,2023-11-13,False,0
1,LDBL69476,0,2023-11-13 12:31:28.615487,2023-11-13,False,0
2,EDGX70669,0,2023-11-13 12:31:28.615487,2023-11-13,False,0
3,VFWO70372,0,2023-11-13 12:31:28.615487,2023-11-13,False,0
4,GLMK71067,0,2023-11-13 12:31:28.615487,2023-11-13,False,0
...,...,...,...,...,...,...
5955,GLMK71067,0,2023-11-02 01:00:00.000000,2023-11-02,False,0
5956,VFWO70372,0,2023-11-02 01:00:00.000000,2023-11-02,False,0
5957,EDGX70669,0,2023-11-02 01:00:00.000000,2023-11-02,False,0
5958,KYGE71098,0,2023-11-02 01:00:00.000000,2023-11-02,False,0


In [94]:
df_stock_days['days_available'].value_counts()

days_available
0     3390
10    2410
9       80
2       40
8       30
1       10
Name: count, dtype: int64

In [96]:
# verificando dados
valor_desejado = 10
linhas_selecionadas = df_stock_days[df_stock_days['days_available'] == valor_desejado]

linhas_selecionadas

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available
13,MENA66694,1,2023-11-13 12:31:28.615487,2023-11-13,True,10
14,MHXR70863,3,2023-11-13 12:31:28.615487,2023-11-13,True,10
15,ELKX63703,6,2023-11-13 12:31:28.615487,2023-11-13,True,10
16,CMXV62869,8,2023-11-13 12:31:28.615487,2023-11-13,True,10
36,SPKV63227,7,2023-11-13 12:31:28.615487,2023-11-13,True,10
...,...,...,...,...,...,...
5923,CMXV62869,9,2023-11-02 01:00:00.000000,2023-11-02,True,10
5925,ELKX63703,6,2023-11-02 01:00:00.000000,2023-11-02,True,10
5945,MHXR70863,3,2023-11-02 01:00:00.000000,2023-11-02,True,10
5946,MENA66694,1,2023-11-02 01:00:00.000000,2023-11-02,True,10


In [154]:
df_mean = df_stock_days.copy()

In [155]:
total_quantity_by_id = df_mean.groupby('ml_inventory_id')['available_quantity'].sum().reset_index()
total_quantity_by_id
# # Fusiona el DataFrame original con la suma total por ml_inventory_id
df_mean = pd.merge(df_mean, total_quantity_by_id, on='ml_inventory_id', how='left')

# # Renombra la columna resultante
df_mean.rename(columns={'available_quantity_x': 'available_quantity'}, inplace=True)
df_mean.rename(columns={'available_quantity_y': 'total_available_quantity'}, inplace=True)
df_mean

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available,total_available_quantity
0,UCQQ19395,0,2023-11-13 12:31:28.615487,2023-11-13,False,0,0
1,LDBL69476,0,2023-11-13 12:31:28.615487,2023-11-13,False,0,0
2,EDGX70669,0,2023-11-13 12:31:28.615487,2023-11-13,False,0,0
3,VFWO70372,0,2023-11-13 12:31:28.615487,2023-11-13,False,0,0
4,GLMK71067,0,2023-11-13 12:31:28.615487,2023-11-13,False,0,0
...,...,...,...,...,...,...,...
5955,GLMK71067,0,2023-11-02 01:00:00.000000,2023-11-02,False,0,0
5956,VFWO70372,0,2023-11-02 01:00:00.000000,2023-11-02,False,0,0
5957,EDGX70669,0,2023-11-02 01:00:00.000000,2023-11-02,False,0,0
5958,KYGE71098,0,2023-11-02 01:00:00.000000,2023-11-02,False,0,0


In [156]:
x = df_mean[df_mean['ml_inventory_id'] == 'RBUB53192']
x

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available,total_available_quantity
201,RBUB53192,17,2023-11-13 12:31:28.615487,2023-11-13,True,10,152
796,RBUB53192,15,2023-11-10 20:22:52.890804,2023-11-10,True,10,152
1589,RBUB53192,15,2023-11-09 16:33:03.451375,2023-11-09,True,10,152
2185,RBUB53192,15,2023-11-08 01:00:00.000000,2023-11-08,True,10,152
2388,RBUB53192,15,2023-11-07 01:00:00.000000,2023-11-07,True,10,152
3003,RBUB53192,15,2023-11-06 01:00:00.000000,2023-11-06,True,10,152
3580,RBUB53192,15,2023-11-05 01:00:00.000000,2023-11-05,True,10,152
4571,RBUB53192,15,2023-11-04 01:00:00.000000,2023-11-04,True,10,152
4773,RBUB53192,15,2023-11-03 01:00:00.000000,2023-11-03,True,10,152
5959,RBUB53192,15,2023-11-02 01:00:00.000000,2023-11-02,True,10,152


In [157]:
x = df_mean[df_mean['ml_inventory_id'] == ml_id_test]
x

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available,total_available_quantity
13,MENA66694,1,2023-11-13 12:31:28.615487,2023-11-13,True,10,10
609,MENA66694,1,2023-11-10 20:22:52.890804,2023-11-10,True,10,10
1775,MENA66694,1,2023-11-09 16:33:03.451375,2023-11-09,True,10,10
2370,MENA66694,1,2023-11-08 01:00:00.000000,2023-11-08,True,10,10
2946,MENA66694,1,2023-11-07 01:00:00.000000,2023-11-07,True,10,10
3542,MENA66694,1,2023-11-06 01:00:00.000000,2023-11-06,True,10,10
4134,MENA66694,1,2023-11-05 01:00:00.000000,2023-11-05,True,10,10
4754,MENA66694,1,2023-11-04 01:00:00.000000,2023-11-04,True,10,10
5327,MENA66694,1,2023-11-03 01:00:00.000000,2023-11-03,True,10,10
5946,MENA66694,1,2023-11-02 01:00:00.000000,2023-11-02,True,10,10


## Calculando se envia

In [206]:
df_calc = df_mean.copy()
print(df_calc.shape)
df_calc.head()

(5960, 7)


,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available,total_available_quantity
0,UCQQ19395,0,2023-11-13 12:31:28.615487,2023-11-13,False,0,0
1,LDBL69476,0,2023-11-13 12:31:28.615487,2023-11-13,False,0,0
2,EDGX70669,0,2023-11-13 12:31:28.615487,2023-11-13,False,0,0
3,VFWO70372,0,2023-11-13 12:31:28.615487,2023-11-13,False,0,0
4,GLMK71067,0,2023-11-13 12:31:28.615487,2023-11-13,False,0,0


In [196]:
## pegando as linhas mais recentes, ultimo dia ##

# indices_linhas_recentes = df_calc.groupby('ml_inventory_id')['created_at'].idxmax()
# indices_linhas_recentes = df_calc.groupby('ml_inventory_id')['data'].idxmax()
# Obtenha os índices das linhas mais recentes para cada 'ml_inventory_id'
indices_linhas_recentes = df_calc.groupby('ml_inventory_id')['created_at'].idxmax()

# Crie o novo DataFrame 'df_hoje' apenas com as linhas mais recentes
df_hoje = df_calc.loc[indices_linhas_recentes].copy()
col_names = {'available_quantity':'', '':'', '':'', '':'', '':'',}
# df_hoje = df_calc.loc[indices_linhas_recentes]
df_hoje = df_hoje.drop(['created_at','has_stock'], axis=1)
df_hoje

,ml_inventory_id,available_quantity,data,days_available,total_available_quantity
189,ABCB20467,0,2023-11-13,0,0
55,ABCB48480,0,2023-11-13,0,0
104,ACBY18022,0,2023-11-13,0,0
476,ADJX46855,0,2023-11-13,0,0
565,ADRN17806,0,2023-11-13,0,0
...,...,...,...,...,...
379,ZXJO91694,7,2023-11-13,10,70
159,ZZGN21450,4,2023-11-13,10,32
358,ZZQL21216,5,2023-11-13,10,59
151,ZZRV30730,7,2023-11-13,10,70


In [174]:
x = df_calc[df_calc['ml_inventory_id'] =='ZZGN21450']
x

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available,total_available_quantity
159,ZZGN21450,4,2023-11-13 12:31:28.615487,2023-11-13,True,10,32
754,ZZGN21450,4,2023-11-10 20:22:52.890804,2023-11-10,True,10,32
1630,ZZGN21450,3,2023-11-09 16:33:03.451375,2023-11-09,True,10,32
2226,ZZGN21450,3,2023-11-08 01:00:00.000000,2023-11-08,True,10,32
2819,ZZGN21450,3,2023-11-07 01:00:00.000000,2023-11-07,True,10,32
3417,ZZGN21450,3,2023-11-06 01:00:00.000000,2023-11-06,True,10,32
3989,ZZGN21450,3,2023-11-05 01:00:00.000000,2023-11-05,True,10,32
4609,ZZGN21450,3,2023-11-04 01:00:00.000000,2023-11-04,True,10,32
5201,ZZGN21450,3,2023-11-03 01:00:00.000000,2023-11-03,True,10,32
5801,ZZGN21450,3,2023-11-02 01:00:00.000000,2023-11-02,True,10,32


In [175]:
df_hoje.columns

Index(['ml_inventory_id', 'available_quantity', 'data', 'days_available',
       'total_available_quantity'],
      dtype='object')

In [202]:
dfx = df_hoje.copy()

In [203]:
days = 30
dfx['media'] = (dfx['total_available_quantity'] / dfx['days_available'])
dfx['30_send_fulfillment'] = np.ceil((dfx['total_available_quantity'] / dfx['days_available'] * days) - dfx['available_quantity'])
dfx['total_send_fulfillment'] = np.ceil((dfx['total_available_quantity'] / dfx['days_available']) - dfx['available_quantity'])


dfx.to_csv('envio.xlsx')
dfx.head(15)

,ml_inventory_id,available_quantity,data,days_available,total_available_quantity,media,30_send_fulfillment,total_send_fulfillment
189,ABCB20467,0,2023-11-13,0,0,NaN,NaN,NaN
55,ABCB48480,0,2023-11-13,0,0,NaN,NaN,NaN
104,ACBY18022,0,2023-11-13,0,0,NaN,NaN,NaN
476,ADJX46855,0,2023-11-13,0,0,NaN,NaN,NaN
565,ADRN17806,0,2023-11-13,0,0,NaN,NaN,NaN
260,AGDL48856,0,2023-11-13,0,0,NaN,NaN,NaN
58,AGDL67534,2,2023-11-13,10,20,2.0,58.0,0.0
69,AGDL67547,0,2023-11-13,0,0,NaN,NaN,NaN
289,AIRQ38676,10,2023-11-13,10,100,10.0,290.0,0.0
60,AIRQ56994,0,2023-11-13,0,0,NaN,NaN,NaN


In [205]:
x = df_calc[df_calc['ml_inventory_id'] == 'AJZG23550']
x

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available,total_available_quantity
304,AJZG23550,9,2023-11-13 12:31:28.615487,2023-11-13,True,10,90
902,AJZG23550,9,2023-11-10 20:22:52.890804,2023-11-10,True,10,90
1486,AJZG23550,9,2023-11-09 16:33:03.451375,2023-11-09,True,10,90
2085,AJZG23550,9,2023-11-08 01:00:00.000000,2023-11-08,True,10,90
2677,AJZG23550,9,2023-11-07 01:00:00.000000,2023-11-07,True,10,90
3274,AJZG23550,9,2023-11-06 01:00:00.000000,2023-11-06,True,10,90
3869,AJZG23550,9,2023-11-05 01:00:00.000000,2023-11-05,True,10,90
4464,AJZG23550,9,2023-11-04 01:00:00.000000,2023-11-04,True,10,90
5062,AJZG23550,9,2023-11-03 01:00:00.000000,2023-11-03,True,10,90
5656,AJZG23550,9,2023-11-02 01:00:00.000000,2023-11-02,True,10,90


In [178]:
dfx['total_send_fulfillment'].value_counts()

total_send_fulfillment
 0.0     147
 1.0      49
 2.0      15
 3.0      11
 4.0       6
 5.0       4
-7.0       3
 8.0       3
-1.0       3
 7.0       2
-2.0       2
-4.0       1
-10.0      1
-3.0       1
-8.0       1
 24.0      1
 6.0       1
-13.0      1
-6.0       1
 14.0      1
-5.0       1
-12.0      1
 79.0      1
Name: count, dtype: int64

In [182]:
x = dfx[dfx['total_send_fulfillment'] == 14]
x

,ml_inventory_id,available_quantity,data,days_available,total_available_quantity,total_send_fulfillment
561,ELQK04205,129,2023-11-13,10,1426,14.0


In [184]:
x = df_calc[df_calc['ml_inventory_id'] == 'ELQK04205']
x

,ml_inventory_id,available_quantity,created_at,data,has_stock,days_available,total_available_quantity
561,ELQK04205,129,2023-11-13 12:31:28.615487,2023-11-13,True,10,1426
1141,ELQK04205,137,2023-11-10 20:22:52.890804,2023-11-10,True,10,1426
1247,ELQK04205,145,2023-11-09 16:33:03.451375,2023-11-09,True,10,1426
1809,ELQK04205,145,2023-11-08 01:00:00.000000,2023-11-08,True,10,1426
2439,ELQK04205,145,2023-11-07 01:00:00.000000,2023-11-07,True,10,1426
3036,ELQK04205,145,2023-11-06 01:00:00.000000,2023-11-06,True,10,1426
3632,ELQK04205,145,2023-11-05 01:00:00.000000,2023-11-05,True,10,1426
4224,ELQK04205,145,2023-11-04 01:00:00.000000,2023-11-04,True,10,1426
4824,ELQK04205,145,2023-11-03 01:00:00.000000,2023-11-03,True,10,1426
5416,ELQK04205,145,2023-11-02 01:00:00.000000,2023-11-02,True,10,1426


In [185]:
(8*145)+137+129

1426